In [16]:
import numpy as np
import json
import pandas as pd
from transformers import pipeline
import os
import torch
import torchvision
from transformers import BertTokenizer, BertForSequenceClassification, AutoModelWithLMHead, PegasusTokenizer, PegasusForConditionalGeneration, TFPegasusForConditionalGeneration
from torchmetrics.text.rouge import ROUGEScore
from transformers import Trainer, TrainingArguments, PegasusForCausalLM
from transformers import AutoTokenizer, AutoModelWithLMHead
from sklearn.model_selection import train_test_split
import evaluate

In [17]:
# rouge = ROUGEScore()

In [18]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [19]:
data = pd.read_csv('train.csv')
df_train = pd.DataFrame(data)
df_train = df_train.drop(['Unnamed: 0'], axis=1)
data_test = pd.read_csv('test.csv')
df_test = pd.DataFrame(data_test)
df_test = df_test.drop(['Unnamed: 0'], axis=1)

In [20]:
df_test.head()

,id,article,stock,date
0,1,Canadian equities resumed their downward trend...,SHOP,2022-07-06
1,2,Destiny Wealth Partners LLC bought a new stake...,CARR,2022-07-03
2,3,"East West Bancorp, Inc. (NASDAQ:EWBC – ) – Inv...",EWBC,2022-07-25
3,4,Community Trust & Investment Co. increased its...,APH,2022-07-06
4,5,Valley Brook Capital Group Inc. lessened its s...,TJX,2022-07-19


In [28]:
#Model for summarization
tokenizer1 = AutoTokenizer.from_pretrained('t5-base')
model1 = AutoModelWithLMHead.from_pretrained('t5-base', return_dict=True)
def summ(article):
    inputs = tokenizer1.encode("summarize: " + article, return_tensors='pt', max_length=80, truncation=True)
    summary_ids = model1.generate(inputs, max_length=80, min_length=80, length_penalty=5., num_beams=2)
    summary = tokenizer1.decode(summary_ids[0])
    return summary

In [22]:
# Model for stock Movement prediction
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
finpred = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

In [23]:
df_pred = df_test
df_pred.head()
# df_test
# len(df_pred['id'])

,id,article,stock,date
0,1,Canadian equities resumed their downward trend...,SHOP,2022-07-06
1,2,Destiny Wealth Partners LLC bought a new stake...,CARR,2022-07-03
2,3,"East West Bancorp, Inc. (NASDAQ:EWBC – ) – Inv...",EWBC,2022-07-25
3,4,Community Trust & Investment Co. increased its...,APH,2022-07-06
4,5,Valley Brook Capital Group Inc. lessened its s...,TJX,2022-07-19


In [24]:
summaries = []
for i in range(5):
    article = df_test['article'][i]
    summaries.append(summ(article))
#     if i%100==0:
    print(i)
# df['summary'] = summaries

0
1
2
3
4


In [25]:
finpred(summaries)

[{'label': 'Positive', 'score': 0.9980807304382324},
 {'label': 'Neutral', 'score': 0.9995177984237671},
 {'label': 'Positive', 'score': 0.9999977350234985},
 {'label': 'Neutral', 'score': 0.9426292777061462},
 {'label': 'Neutral', 'score': 0.9918287396430969}]

In [26]:
# df_pred.rename(columns = {'article':'summary'}, inplace = True)
df_pred.head()

,id,article,stock,date
0,1,Canadian equities resumed their downward trend...,SHOP,2022-07-06
1,2,Destiny Wealth Partners LLC bought a new stake...,CARR,2022-07-03
2,3,"East West Bancorp, Inc. (NASDAQ:EWBC – ) – Inv...",EWBC,2022-07-25
3,4,Community Trust & Investment Co. increased its...,APH,2022-07-06
4,5,Valley Brook Capital Group Inc. lessened its s...,TJX,2022-07-19


In [29]:
for i in range(1):
    summary = df_pred['summary'][i]
    finpredic = finpred(summary)
finpredic

In [31]:
summaries[1]

'<pad> Destiny Wealth Partners LLC bought 1,300 shares of the company’s stock, valued at approximately $600,000. a number of other hedge funds have also recently added to or reduced their stakes in CARR. a number of other hedge funds have also recently added to or reduced their stakes in CARR. a number of other hedge funds have also recently'